In [ ]:
# Use the environment variable if the user doesn't provide Project ID.
import os

import vertexai
from vertexai.preview.generative_models import (
    FunctionDeclaration,
    GenerativeModel,
    Tool,
    ToolConfig,
    Part,
    GenerationConfig,
)
PROJECT_ID = "104916006626"  # @param {type: "string", placeholder: "[your-project-id]" isTemplate: true}
if not PROJECT_ID or PROJECT_ID == "xyz":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "australia-southeast1")

vertexai.init(project="104916006626", location=LOCATION)

In [ ]:
import os
from google.oauth2 import service_account

# Path to your service account key file
key_path = "C:\\Users\\shres\\Projects\\RAG-case-study\keys\\keyproject-401005-6e1cdcbb5996.json"

# Create credentials using the service account key file
credentials = service_account.Credentials.from_service_account_file(
    key_path
)

# Set the credentials for the current environment
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = key_path
# auth_request = transport.requests.Request()
# credentials.refresh(auth_request)

In [ ]:
# Controlled generation example test
response_schema = {
    "type": "array",
    "items": {
        "type": "object",
        "properties": {
            "recipe_name": {
                "type": "string",
            },
        },
        "required": ["recipe_name"],
    },
}

model = GenerativeModel("gemini-1.5-pro-002")

response = model.generate_content(
    "List a few popular cookie recipes",
    generation_config=GenerationConfig(
        response_mime_type="application/json", response_schema=response_schema
    ),
)

print(response.text)

## Document preprocessing:

Each document is to pre-processed individually as the structure of each document is different from the other. The objective it to extract meaningful chunks out of each document while preserving the original structure of the document, long term and short term context so that the chunk is indexed better and retrieved accurately.
The chunking strategy is as follows:
- LLM to create an index for the document dividing the document into various sections based on heading and sub headings. This will be used as metadata for the chunk.
- The document generates chunks by outputing the first and last sentence for the chunk. The chunk is then extracted from the md file which contains the document text. Generated chunk is restricted insize to a set number of tokens
- Each chunk is appended with short term and long term context for the document to aid the embedding and retrieval process.
- LLM must help remove unnecessary headers and footers  of the document
